In [1]:
from gen_landscape import *
from world_class_v2 import *

In [60]:
land = Landscape(2, 300, num_grid_res=2)

In [ ]:
def create_world_and_run_till_end(sex_win, asex_win, land):
    gia = world(population_size=10000, loci=2, gene_mean=150, gene_sd=10, proportion_asexual=0.5, survival_rate=0.7,
                asex_repl_ratio=10 / 7, sex_repl_ratio=10 / 7, mutation_down_prob=0.05, mutation_up_prob=0.05,
                mutation_step=1, control=4, landscape=land.fitness_grid)
    
    print('max for each loci', np.max(gia.total_pop_mat, axis = 1))
    print('min for each loci:', np.min(gia.total_pop_mat, axis = 1))
    for i in range(100000):
        if gia.population_sizes(asex=True) < 100:
            sex_win += 1
            print('SEX WINNER')
            break
        elif gia.population_sizes(sex=True) < 100:
            asex_win += 1
            print('ASEX WINNER')
            break
        gia.mutation_stage()
        gia.survival_stage()
        gia.replication_stage()
        if i % 500 == 0:
            print('Iteration:', i)
            print('Asex population percentage:', gia.population_sizes(asex=True) / gia.population_sizes(total=True))
    return sex_win, asex_win

sex_win, asex_win = 0, 0
land_num = 0
land_dict = {}
for n in range(10):
    land = Landscape(2, 300, num_grid_res=2)
    sex_win, asex_win = create_world_and_run_till_end(sex_win, asex_win, land)

print('INTER-ITER: Sex:', sex_win, ' Asex:', asex_win)
print('iterations', i)

max for each loci [188 189]
min for each loci: [114 115]
Iteration: 0
Asex population percentage: 0.4987
Iteration: 500
Asex population percentage: 0.9356
ASEX WINNER
max for each loci [187 191]
min for each loci: [111 110]
Iteration: 0
Asex population percentage: 0.5064
Iteration: 500
Asex population percentage: 0.2164
SEX WINNER
max for each loci [188 191]
min for each loci: [104 111]
Iteration: 0
Asex population percentage: 0.4993
Iteration: 500
Asex population percentage: 0.0629
SEX WINNER
max for each loci [189 189]
min for each loci: [110 111]
Iteration: 0
Asex population percentage: 0.5027
Iteration: 500
Asex population percentage: 0.2427
Iteration: 1000
Asex population percentage: 0.239
Iteration: 1500
Asex population percentage: 0.026
SEX WINNER
max for each loci [184 187]
min for each loci: [111 111]
Iteration: 0
Asex population percentage: 0.4994
